In [1]:
# Sample Code Provided by Southern New Hampshire University Resources

# Title: Project 2 Submission
# Edited By: Joel De Alba (SNHU Student)
# Institution: Southern New Hampshire University
# Date: 6/19/23
# Professor Dr. Tad Kellog

# Importing Dash Core for UI Implementation
from jupyter_plotly_dash import JupyterDash

# Necessary dash functionality Imports
import dash
import dash_leaflet as dl
import plotly.express as px
from dash import html
from dash import dcc
from dash import dash_table as dt
from dash.dependencies import Input, Output, State

# Other necessary imports such as numpy for scientific computing, and pandas for graphical data structures
import os
import numpy as np
import pandas as pd

# For Binary to Text conversions
import base64

# MongoB import
from pymongo import MongoClient
from bson.json_util import dumps

# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Username and Password Set
username = "accuser"
password = "SNHU1234"

# Variable that sets AnimalShelter username and password based on variables
shelter = AnimalShelter(username, password)

# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Grazioso Salvare’s logo file name implemented
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Image and Unique identifier implemented
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Center(html.B(html.H2('SNHU CS-340 Dashboard - Joel De Alba'))),
    html.Hr(),
    html.Div(
        

    className='row',
    style={'display': 'flex'},

    children=[
    html.Button(id='button-one',n_clicks=0, children= 'Water Rescue'),
    html.Button(id='button-two',n_clicks=0, children= 'Mountain or Wilderness Rescue'),
    html.Button(id='button-three',n_clicks=0, children='Disaster Rescue or Tracking'),
    html.Button(id='button-four', n_clicks=0, children='Cats'),
    html.Button(id='button-five', n_clicks=0, children='Dogs'),
    html.Button(id='button-six', n_clicks=0, children='Reset Data')
    ]


    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),

        
        # User friendly features implemented
        page_size=100,
        style_table={
            'height':'400px',
            'overflowY':'auto',
            'overflowX':'auto'
        },

        style_header={
            'backgroundColor':'rgb(240,230,230)',
            'fontWeight':'bold'
        },

        style_data={
            'whiteSpace':'normal',
            'height':'auto'
        },



        # Tooltip implementation
        tooltip ={i: {
            'value': i,
            'use_with': 'both'
        } for i in df.columns},

        tooltip_delay=0,
        tooltip_duration = None,
        sort_action='native',
        sort_mode='multi',
        filter_action='native',
        editable=False,
        column_selectable=True,
        row_selectable='single',
        row_deletable=False,
        selected_rows=[],
    ),
    
    html.Br(),
    html.Hr(),
    
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
    
@app.callback([Output('datatable-id','data')],
            [Input('button-one', 'n_clicks'),Input('button-two','n_clicks'),
            Input('button-three','n_clicks'),Input('button-four','n_clicks'),
            Input('button-five', 'n_clicks'),Input('button-six', 'n_clicks')])

def update_dashboard(button1,button2,button3,button4,button5,button6):

        # Interactive Table Implementation
        if (int(button1) >= 1):
            df = pd.DataFrame.from_records(shelter.read({'$and': [
            {'$or': [ {'breed':'Labrador Retriever Mix'}, {'breed':'Chesapeake Bay Retriever'},
            {'breed':'Newfoundland'}]}, {'sex_upon_outcome':'Intact Female'}, 
            {'age_upon_outcome_in_weeks':{'$lte':26, 'gte':156}}]}))
            button2, button3, bt4, button5, button6 = 0

        elif (int(button2)>= 1):
            df = pd.DataFrame.from_records(shelter.read({'$and': [
            {'$or': [ {'breed':'German Shepherd'}, {'breed':'Alaskan Malamute'},
            {'breed':'Old English Sheepdog'},{'breed':'Siberian Husky'},{'breed':'Rottweiler'}]},
            {'sex_upon_outcome':'Intact Male'}, {'age_upon_outcome_in_weeks':{'$lte':26, 'gte':156}}]}))
            button1, button3 , bt4, button5, button6  = 0

        elif (int(button3)>=1):
            df = pd.DataFrame.from_records(shelter.read({'$and': [
            {'$or': [ {'breed':'Doberman Pinscher'}, {'breed':'German Sheperd'},
            {'breed':'Golden Retriever'},{'breed':'Bloodhound'},{'breed':'Rottweiler'}]},
            {'sex_upon_outcome':'Intact Male'}, {'age_upon_outcome_in_weeks':{'$lte':20, 'gte':300}}]}))
            bt1, button2, bt4, button5, button6 = 0
                               
        elif(int(button4)>=1):
            df = pd.DataFrame(db.read({"animal_type":"Cat"}))
            bt1, button2, button3, button5, button6 = 0
                               
        elif(int(button5)>=1):
            df = pd.DataFrame(db.read({"animal_type":"Dog"}))
            bt1, button2, button3, button4, button6 = 0
                               
        elif(int(button6)>=1):
            df = pd.DataFrame.from_records(shelter.read())
            bt1, button2, button3, button4, button5 = 0

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        return (data, columns)


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    df = pd.DataFrame.from_dict(viewData)
    return [
    dcc.Graph(figure = px.pie(df, values=values, names=names, title='Available Breeds by percentage')
)
]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    # Geolocation Implemented
    dff = pd.DataFrame.from_dict(viewData)
    return [ dl.Map(style={
        'width': '1000px', 
        'height': '500px'}, 
        center=[dff.loc[0,'location_lat'],
        dff.loc[0,'location_long']], zoom=15, 
        children=[dl.TileLayer(id="base-layer-id"),
        dl.Marker(position=[dff.loc[0,'location_lat'],
        dff.loc[0,'location_long']], 
        children=[dl.Tooltip(dff['breed']),
        dl.Popup([html.H1("Animal Name"),
        html.P(dff.loc[0,'name'])
        ])
    ])
])]

app

TypeError: object of type 'NoneType' has no len()